In [16]:
# Librerías
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.m_estimate import MEstimateEncoder
from scipy import stats
from scipy.stats import norm, probplot
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [17]:
# Cargar datos limpios
train_data_num = pd.read_csv("../Data/clean/num_clean.csv")
train_data_cat = pd.read_csv("../Data/clean/cat_clean.csv")

### Estandaricemos las características en train_data_num. Esto mejora la eficacia de los modelos de ML.

In [18]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Aplicamos StandardScaler a las columnas numéricas
scaler = StandardScaler()
train_data_num = pd.DataFrame(scaler.fit_transform(train_data_num), columns=train_data_num.columns)
train_data_num.head()

,OverallCond,latest_contruction,TotalBsmtSF,GrLivArea,GarageArea,SalePrice,indicator
0,-0.507284,0.896804,-0.444278,0.413547,0.348888,0.347273,0.999657
1,2.188279,-0.395615,0.477158,-0.471891,-0.059804,0.007288,0.999657
2,-0.507284,0.848937,-0.299027,0.563755,0.627542,0.536154,0.999657
3,-0.507284,-0.682820,-0.671232,0.427382,0.785446,-0.515281,0.999657
4,-0.507284,0.753202,0.211621,1.378042,1.686426,0.869843,0.999657


# Variables no numéricas

In [19]:
# Initialize df_stats with specified columns which indicate the metrics for analysis
df_stats = pd.DataFrame(columns=['column', 'Distinct_value_incl_na', 'Distinct_value_without_na', 
                                 'missing_val', '%_missing_val'])

# List to hold the data for each column
stats = []

for c in train_data_cat.columns:
    column_stats = {
        'column': c,
        'Distinct_value_incl_na': len(list(train_data_cat[c].unique())),
        'Distinct_value_without_na': int(train_data_cat[c].nunique()),
        'missing_val': train_data_cat[c].isnull().sum(),
        '%_missing_val': (train_data_cat[c].isnull().sum() / len(train_data_cat)).round(3) * 100
    }
    stats.append(column_stats)

# Convert the list of dictionaries to a DataFrame
df_stats = pd.DataFrame(stats)
df_stats.head()

,column,Distinct_value_incl_na,Distinct_value_without_na,missing_val,%_missing_val
0,MSZoning,6,5,4,0.1
1,Street,2,2,0,0.0
2,LotShape,4,4,0,0.0
3,LandContour,4,4,0,0.0
4,Utilities,3,2,2,0.1


In [20]:
# Impute the features before encoding it.

def mode_imputation(train_data_cat):
 
    for col in train_data_cat.columns:
        mode = train_data_cat[col].mode().iloc[0]
        train_data_cat[col] = train_data_cat[col].fillna(mode)
    return train_data_cat

train_data_cat = mode_imputation(train_data_cat)

In [21]:
nominal_cols = ['MSZoning', 'Street','LandContour','Neighborhood','Condition1','Condition2',
                'RoofStyle','RoofMatl','Exterior1st','Exterior2nd','Heating','GarageType','SaleType','SaleCondition']
ordinal_cols = ['ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',
                'BsmtFinType2','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','GarageFinish','GarageQual','GarageCond','PavedDrive','LotShape',
                'Utilities','LandSlope','BldgType','HouseStyle','LotConfig']


### Se dividen los datos categóricos en 2: cardinales y nominales.

In [22]:
train_ordinal = train_data_cat[ordinal_cols]
train_nominal = train_data_cat[nominal_cols]

In [23]:
# Let's label encode the ordinal data
def label_encode(train_ordinal):
    """
    Label encoding of the categorical features
    """
    '''Create a copy of train_ordinal'''
    train_ordinal_encoded = train_ordinal.copy()
    lab_enc_dict = {}
    for col in train_ordinal_encoded:
        lab_enc_dict[col] = LabelEncoder()
        train_ordinal_encoded[col] = lab_enc_dict[col].fit_transform(train_ordinal[col])
    return train_ordinal_encoded

train_ordinal_encoded = label_encode(train_ordinal)
train_ordinal_encoded.head()

,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,CentralAir,Electrical,...,GarageFinish,GarageQual,GarageCond,PavedDrive,LotShape,Utilities,LandSlope,BldgType,HouseStyle,LotConfig
0,2,4,2,3,3,2,5,0,1,4,...,1,4,4,2,3,0,0,0,5,4
1,3,4,2,3,1,0,5,0,1,4,...,1,4,4,2,3,0,0,0,2,2
2,2,4,2,3,2,2,5,0,1,4,...,1,4,4,2,0,0,0,0,5,4
3,3,4,3,1,3,0,5,2,1,4,...,2,4,4,2,0,0,0,0,5,0
4,2,4,2,3,0,2,5,0,1,4,...,1,4,4,2,0,0,0,0,5,2


### *One-hot encoding* para datos ordinales.

In [24]:
# Let's do one-hot encoding for the nominal data
def onehot_encode(train_nominal):
    train_onehot_encoded = pd.get_dummies(train_nominal[train_nominal.columns[:-1]])
    return train_onehot_encoded

train_nominal_onehot_encoded = onehot_encode(train_nominal)

# If the encoded DataFrame contains True/False, convert them to 0/1
train_nominal_onehot_encoded = train_nominal_onehot_encoded.astype(int)
train_nominal_onehot_encoded.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LandContour_Bnk,LandContour_HLS,LandContour_Low,...,GarageType_Detchd,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,1,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [25]:
total = train_data_cat.isnull().sum().sort_values(ascending=False)
percent = (train_data_cat.isnull().sum()*100/train_data_cat.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, 100-percent], axis=1, keys=['Total', 'Fill rate of the Features'])
missing_data.head(20)

,Total,Fill rate of the Features
MSZoning,0,100.0
Electrical,0,100.0
BsmtCond,0,100.0
BsmtExposure,0,100.0
BsmtFinType1,0,100.0
BsmtFinType2,0,100.0
Heating,0,100.0
HeatingQC,0,100.0
CentralAir,0,100.0
KitchenQual,0,100.0


### Vamos a concatenar los datos categóricos para las características ordinales y nominales. Después de la concatenación, aplicar estandarización.

In [26]:
# Concatenate
train_data_cat = pd.concat([train_ordinal_encoded, train_nominal_onehot_encoded], axis=1)

# Applying StandardScaler 
scaler = StandardScaler()
train_data_cat = pd.DataFrame(scaler.fit_transform(train_data_cat), columns=train_data_cat.columns)
train_data_cat.head()

,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,CentralAir,Electrical,...,GarageType_Detchd,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,-0.755793,0.376461,-0.294203,0.28567,0.624607,-0.449736,0.315909,-0.880324,0.26829,0.300615,...,-0.603340,-0.175272,-0.064249,-0.041423,-0.094801,-0.055613,-0.052423,-0.298629,-0.049029,0.394439
1,0.668455,0.376461,-0.294203,0.28567,-1.158500,-1.546918,0.315909,-0.880324,0.26829,0.300615,...,-0.603340,-0.175272,-0.064249,-0.041423,-0.094801,-0.055613,-0.052423,-0.298629,-0.049029,0.394439
2,-0.755793,0.376461,-0.294203,0.28567,-0.266947,-0.449736,0.315909,-0.880324,0.26829,0.300615,...,-0.603340,-0.175272,-0.064249,-0.041423,-0.094801,-0.055613,-0.052423,-0.298629,-0.049029,0.394439
3,0.668455,0.376461,0.835768,-2.68713,0.624607,-1.546918,0.315909,0.267617,0.26829,0.300615,...,1.657441,-0.175272,-0.064249,-0.041423,-0.094801,-0.055613,-0.052423,-0.298629,-0.049029,0.394439
4,-0.755793,0.376461,-0.294203,0.28567,-2.050054,-0.449736,0.315909,-0.880324,0.26829,0.300615,...,-0.603340,-0.175272,-0.064249,-0.041423,-0.094801,-0.055613,-0.052423,-0.298629,-0.049029,0.394439


### Vamos a concatenar los dataframes de datos numéricos y datos categóricos.

In [ ]:
# Nuevos dataframes
salesprice = pd.DataFrame()
indicator = pd.DataFrame()

salesprice = train_data['SalePrice']
indicator = train_data['is_train']

In [35]:
# Concatenate the numerical and categorical dataframes and add the flag for train-test data and also the target variable.
train_data_v1 = pd.concat([train_data_num, train_data_cat], axis=1)
#train_data_v1['SalePrice'] = salesprice
#train_data_v1['indicator'] = indicator
train_data_v1.columns

Index(['OverallCond', 'latest_contruction', 'TotalBsmtSF', 'GrLivArea',
       'GarageArea', 'SalePrice', 'indicator', 'ExterQual', 'ExterCond',
       'BsmtQual',
       ...
       'GarageType_Detchd', 'SaleType_COD', 'SaleType_CWD', 'SaleType_Con',
       'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New',
       'SaleType_Oth', 'SaleType_WD'],
      dtype='object', length=148)

In [28]:
# Let's eliminate the flag column after the data segregation
train_data_v2 = train_data_v1[train_data_v1['indicator'] == 1].drop(columns=['indicator'])
test_data_v2 = train_data_v1[train_data_v1['indicator'] == 0].drop(columns=['indicator'])

In [29]:
train_data_v2.head()

,OverallCond,latest_contruction,TotalBsmtSF,GrLivArea,GarageArea,SalePrice,ExterQual,ExterCond,BsmtQual,BsmtCond,...,GarageType_Detchd,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD


In [14]:
# Applying log transformation to SalePrice field
train_data_v2['SalePrice'] = np.log(train_data_v2['SalePrice'])

In [15]:
#Exportamos los datos
train_data_v2.to_csv('../Data/clean/train_clean_data.csv', index=False)
test_data_v2.to_csv('../Data/clean/test_clean_data.csv', index=False)